<a href="https://colab.research.google.com/github/yangxuan8/EVSIvsLSTM/blob/main/run_baseline_26_02_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive')

Mounted at /content/drive


In [2]:
!pip install pytorch_lightning
import torch
import pickle
import argparse
import numpy as np
import torch.nn as nn
from torchmetrics import AUROC, Accuracy
from torch.utils.data import DataLoader, random_split, TensorDataset
import pytorch_lightning as pl
import pandas as pd
sys.path.append('/content/drive/MyDrive/DIME_main/experiments/hev')
import feature_groups
sys.path.append('/content/drive/MyDrive/DIME_main')
import dime
from dime.data_utils import HEVDataset, get_group_matrix, get_xy
from dime import MaskingPretrainer
from dime.utils import StaticMaskLayer1d, ConcreteMask, get_confidence, MaskLayerGrouped, get_mlp_network
import torch.optim as optim
from tqdm import tqdm
from baseline_models.base_model import BaseModel
sys.path.append('/content/drive/MyDrive/DIME_main/experiments')
from baselines import eddi, pvae, iterative, dfs, cae

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 39.0 MB/s eta 0:00:00


In [3]:
# Set up command line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--method', type=str, default='dime', choices=['cae', 'eddi', 'dfs', 'dime', 'fully_supervised'])
parser.add_argument('--use_feature_costs', default=False, action="store_true")
parser.add_argument('--num_trials', type=int, default=3)

_StoreAction(option_strings=['--num_trials'], dest='num_trials', nargs=None, const=None, default=3, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [5]:
if __name__ == '__main__':
    hev_feature_names = feature_groups.hev_feature_names
    hev_feature_groups = feature_groups.hev_feature_groups
    auc_metric = AUROC(task='multiclass', num_classes=3)
    acc_metric = Accuracy(task='multiclass', num_classes=3)

    # Parse args
    args = parser.parse_known_args()[0]
    device = torch.device('cuda', args.gpu)
    num_trials = args.num_trials
    cols_to_drop = []
    if cols_to_drop is not None:
        hev_feature_names = [item for item in hev_feature_names if str(hev_feature_names.index(item)) not in cols_to_drop]
    # Load dataset
    dataset = HEVDataset(data_dir=1, cols_to_drop=cols_to_drop)
    d_in = dataset.X.shape[1]  # 32
    d_out = len(np.unique(dataset.Y))  # 3

    # Get features and groups
    feature_groups_dict, feature_groups_mask = get_group_matrix(hev_feature_names, hev_feature_groups)
    feature_group_indices = {i : key for i, key in enumerate(feature_groups_dict.keys())}
    feat_to_ind = {key: i for i, key in enumerate(hev_feature_names)}

    num_groups = len(feature_groups_mask)  # 32
    print("Num groups=", num_groups)
    print("Num features=", d_in)

    # Split dataset
    train_dataset, val_dataset, test_dataset = random_split(dataset, [0.8, 0.1, 0.1], generator=torch.Generator().manual_seed(0))
    daataset_dict = dict(train_dataset=train_dataset, val_dataset=val_dataset, test_dataset=test_dataset)
    f = open('/content/drive/MyDrive/dataset/data/dataset.pkl', "wb", pickle.HIGHEST_PROTOCOL)
    pickle.dump(daataset_dict, f)

    print(f'Train samples = {len(train_dataset)}, val samples = {len(val_dataset)}, test samples = {len(test_dataset)}')

    # Find mean/variance for normalizing
    x, y = get_xy(train_dataset)
    mean = np.mean(x, axis=0)
    std = np.clip(np.std(x, axis=0), 1e-3, None)

    # Normalize via the original dataset
    if args.method == 'eddi':
        dataset.X = (dataset.X - mean)/std
    else:
        dataset.X = dataset.X - mean

    # Set up data loaders.
    train_dataloader = DataLoader(
        train_dataset, batch_size=128, shuffle=True, pin_memory=True,
        drop_last=True, num_workers=4)

    val_dataloader = DataLoader(
        val_dataset, batch_size=128, shuffle=False, pin_memory=True, drop_last=True, num_workers=4)

    test_dataloader = DataLoader(
        test_dataset, batch_size=128, shuffle=False, pin_memory=True, drop_last=True, num_workers=4)


    mask_layer = MaskLayerGrouped(append=True, group_matrix=torch.tensor(feature_groups_mask))
    num_features = [1, 3, 5, 10, 15, 20, 25, 30, 32]
    use_feature_costs = False
    feature_costs = None
    if args.use_feature_costs:
        feature_cost_df = pd.read_csv("data/feature_list_hev-nw.csv")
        feature_costs = [feature_cost_df[feature_cost_df['Feature Name'] == feature]['Cost (Hours)'].item() for feature in list(feature_groups_dict.keys())]
        use_feature_costs = True

    for trial in range(num_trials):

        results_dict = {
            'acc': {},
            'features': {}
        }


        if args.method == 'eddi':
            # Train PVAE.
            bottleneck = 16
            hidden = 128
            dropout = 0.3
            encoder = get_mlp_network(d_in + num_groups, bottleneck * 2)
            decoder = get_mlp_network(bottleneck, d_in)

            pv = pvae.PVAE(encoder, decoder, mask_layer, 20, 'gaussian').to(device)
            pv.fit(
                train_dataloader,
                val_dataloader,
                lr=1e-3,
                nepochs=10,
                verbose=False)

            # Train masked predictor.
            model = get_mlp_network(d_in + num_groups, d_out)
            sampler = None
            # if trial == 0:
            sampler = iterative.UniformSampler(get_xy(train_dataset)[0])  # TODO don't actually need sampler
            iterative_selector = iterative.IterativeSelector(model, mask_layer, sampler).to(device)
            iterative_selector.fit(
                train_dataloader,
                val_dataloader,
                lr=1e-3,
                nepochs=10,
                loss_fn=nn.CrossEntropyLoss(),
                patience=5,
                verbose=True)

            # Set up EDDI feature selection object.
            eddi_selector = eddi.EDDI(pv, model, mask_layer, feature_costs=feature_costs).to(device)

            # Evaluate.
            metrics_dict, cost_dict = eddi_selector.evaluate_multiple(test_dataloader, num_features, auc_metric, verbose=False)
            for num in num_features:
                acc = metrics_dict[num]
                results_dict['acc'][num] = acc
                print(f'Num = {num}, Acc = {100*acc:.2f}')

            print(results_dict)
            print(cost_dict)
            with open(f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}_feature_costs_{use_feature_costs}.pkl', 'wb') as f:
                pickle.dump(results_dict, f)

            with open(f'/content/drive/MyDrive/dataset/results/hev_costs_{args.method}_trial_{trial}_feature_costs_{use_feature_costs}.pkl', 'wb') as f:
                pickle.dump(cost_dict, f)

        if args.method == 'dfs':
            max_features = 32

            # Prepare networks.
            predictor = get_mlp_network(d_in + num_groups, d_out)
            selector = get_mlp_network(d_in + num_groups, num_groups)

            # Pretrain predictor

            pretrain = MaskingPretrainer(
                predictor,
                mask_layer,
                lr=1e-3,
                loss_fn=nn.CrossEntropyLoss(),
                val_loss_fn=auc_metric)

            trainer = pl.Trainer(max_epochs=10)
            trainer.fit(pretrain, train_dataloader, val_dataloader)

            """
            pretrain = MaskingPretrainer(predictor, mask_layer,).to(device)

            pretrain.fit(
                train_dataset,
                val_dataset,
                lr=1e-3,
                val_loss_fn=auc_metric,
                val_loss_mode='max',
                nepochs=10,
                loss_fn=nn.CrossEntropyLoss(),
                patience=5,
                verbose=True)
            """


            # Train selector and predictor jointly.
            gdfs = dfs.GreedyDynamicSelection(selector, predictor, mask_layer).to(device)
            gdfs.fit(
                train_dataloader,
                val_dataloader,
                lr=1e-3,
                nepochs=10,
                max_features=max_features,
                loss_fn=nn.CrossEntropyLoss(),
                patience=3,
                verbose=True)

            # Evaluate.
            for num in num_features:
                auroc_list = []
                acc_list = []

                auroc, acc = gdfs.evaluate(test_dataloader, num, (auc_metric, acc_metric))
                #results_dict['acc'][num] = acc
                #print(f'Num = {num}, Acc = {100*acc:.2f}')
                auroc_list.append(auroc)
                acc_list.append(acc)
                print(f'Num = {num}, AUROC = {100*auroc:.2f}, Acc = {100*acc:.2f}')

            with open(f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pkl', 'wb') as f:
                pickle.dump(results_dict, f)

            # Save model
            gdfs.cpu()
            torch.save(gdfs, f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pt')

        if args.method == 'dime':
            max_features = 32

            # Prepare networks.
            hidden = 128
            dropout = 0.3
            predictor = nn.Sequential(
                      nn.Linear(d_in + num_groups, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, d_out))

            selector = nn.Sequential(
                      nn.Linear(d_in + num_groups, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, hidden),
                      nn.ReLU(),
                      nn.Dropout(dropout),
                      nn.Linear(hidden, num_groups))

            # Pretrain predictor

            pretrain = MaskingPretrainer(
                predictor,
                mask_layer,
                lr=1e-3,
                loss_fn=nn.CrossEntropyLoss(),
                val_loss_fn=auc_metric)

            trainer = pl.Trainer(max_epochs=30)
            trainer.fit(pretrain, train_dataloader, val_dataloader)

            # Train selector and predictor jointly.
            print("Training CMI estimator")
            print("-"*8)
            gdfs = dfs.GreedyDynamicSelection(selector, predictor, mask_layer).to(device)
            gdfs.fit(
                train_dataloader,
                val_dataloader,
                lr=1e-3,
                nepochs=30,
                max_features=max_features,
                #eps=0.1,
                loss_fn=nn.CrossEntropyLoss(),
                patience=5,
                #feature_costs=feature_costs,
                verbose=True)

            # Evaluate.
            for num in num_features:
                auroc_list = []
                acc_list = []

                #acc = gdfs.evaluate(test_dataloader, num, auc_metric)
                #results_dict['acc'][num] = acc
                #print(f'Num = {num}, Acc = {100*acc:.2f}')
                auroc, acc = gdfs.evaluate(test_dataloader, num, (auc_metric, acc_metric))
                auroc_list.append(auroc)
                acc_list.append(acc)
                print(f'Num = {num}, AUROC = {100*auroc:.2f}, Acc = {100*acc:.2f}')


            with open(f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pkl', 'wb') as f:
                pickle.dump(results_dict, f)

            # Save model
            gdfs.cpu()
            torch.save(gdfs, f'/content/drive/MyDrive/dataset/results/hev_{args.method}_trial_{trial}.pt')

        # Train with full input
        if args.method == 'fully_supervised':
            model = get_mlp_network(d_in, d_out).to(device)
            opt = optim.Adam(model.parameters(), lr=1e-3)
            criterion = torch.nn.CrossEntropyLoss()
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                    opt, mode='min', factor=0.2, patience=5,
                    min_lr=1e-5, verbose=True)

            num_bad_epochs = 0
            early_stopping_epochs = 6

            for epoch in range(100):
                model.train()
                train_batch_loss = 0
                val_batch_loss = 0
                val_pred_list = []
                val_y_list = []

                for i, (x, y) in enumerate(tqdm(train_dataloader)):
                    x = x.to(device)
                    y = y.to(device)

                    pred = model(x)
                    train_loss = criterion(pred, y)
                    train_batch_loss += train_loss.item()
                    train_loss.backward()
                    opt.step()
                    model.zero_grad()

                model.eval()

                with torch.no_grad():
                    for i, (x, y) in enumerate(tqdm(val_dataloader)):
                        x = x.to(device)
                        y = y.to(device)

                        pred = model(x)
                        val_loss = criterion(pred, y)
                        val_batch_loss += val_loss.item()
                        val_pred_list.append(pred.cpu())
                        val_y_list.append(y.cpu())

                    scheduler.step(val_batch_loss/len(val_dataloader))
                    val_loss = val_batch_loss/len(val_dataloader)
                    # Check if best model.
                    if val_loss == scheduler.best:
                        # best_model = deepcopy(model)
                        num_bad_epochs = 0
                    else:
                        num_bad_epochs += 1

                    # Early stopping.
                    if num_bad_epochs > early_stopping_epochs:
                        print(f'Stopping early at epoch {epoch+1}')
                        break

                print(f"Epoch: {epoch}, Train Loss: {train_batch_loss/len(train_dataloader)},"
                      + f"Val Loss: {val_batch_loss/len(val_dataloader)},"
                      + f"Val Performance: {auc_metric(torch.cat(val_pred_list), torch.cat(val_y_list))}")

            print("Evaluating on test set")

            model.eval()
            confidence_list = []
            test_pred_list = []
            test_y_list = []
            for i, (x, y) in enumerate(tqdm(test_dataloader)):
                x = x.to(device)
                y = y.to(device)

                pred = model(x)
                test_pred_list.append(pred.cpu())
                test_y_list.append(y.cpu())

                confidence_list.append(get_confidence(pred.cpu()))

            print(f"Test Performance:{auc_metric(torch.cat(test_pred_list), torch.cat(test_y_list))}")
            with open('/content/drive/MyDrive/dataset/confidence.npy', 'wb') as f:
                np.save(f, np.array(torch.cat(confidence_list).detach().numpy()))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type             | Params
--------------------------

Index(['faultNumber', 'VelocityRef:1', '<xdot>', '<BattSoc>', '<BattPwr>',
       '<Cltch1State>', '<Cltch2State>', '<BattV>', '<TransGear>', '<EngSpd>',
       '<IntkVlvLift>', '<EngTrq>', '<ThrPosPct>', '<WgAreaPct>',
       '<EgrVlvAreaPct>', '<VarCompRatioPos>', '<Acc>', '<Dec>', '<IgSw>',
       '<Chrg>', 'TransGear', 'BrkCmd', 'Cltch1Cmd', '<MotTrq>', '<StartTrq>',
       'StartCmd', 'MotTrqCmd', 'BattCrnt:1', 'MotPwrElec:1', 'MotPwrMech:1',
       'IntkVlvLiftCmd', 'FuelMainSoi', 'FuelFlw'],
      dtype='object')
Num groups= 32
Num features= 32
Train samples = 9749, val samples = 1219, test samples = 1218


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

100%|██████████| 9/9 [00:00<00:00, 10.50it/s]


--------Epoch 7 (86 total)--------
Val loss = 0.5610, Zero-temp loss = 0.5610

Epoch 00007: reducing learning rate of group 0 to 2.0000e-04.


100%|██████████| 9/9 [00:00<00:00, 10.41it/s]

--------Epoch 8 (87 total)--------
Val loss = 0.5549, Zero-temp loss = 0.5545

Stopping temp = 0.1000 at epoch 8



Num = 1, AUROC = 80.61, Acc = 61.89
Num = 3, AUROC = 82.04, Acc = 64.32
Num = 5, AUROC = 85.60, Acc = 70.05
Num = 10, AUROC = 87.59, Acc = 72.05
Num = 15, AUROC = 90.94, Acc = 77.17
Num = 20, AUROC = 91.68, Acc = 78.91
Num = 25, AUROC = 87.26, Acc = 73.70
Num = 30, AUROC = 84.44, Acc = 67.19
Num = 32, AUROC = 56.92, Acc = 40.10
